In [1]:
!pip install bitarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 3.3 MB/s eta 0:00:00


In [2]:
import math
from struct import pack
from bitarray import bitarray
import numpy as np
import cv2

In [3]:
def Q_l():
  return(np.array([[16,11,10,16,24,40,51,61],[12,12,14,19,26,58,60,55],[14,13,16,24,40,57,69,56],[14,17,22,29,51,87,80,62],[18,22,37,56,68,109,103,77],[24,35,55,64,81,104,113,92],[49,64,78,87,103,121,120,101],[72,92,95,98,112,100,103,99]]))
def Q_c():
  return(np.array([[17,1,24,47,99,99,99,99],[18,21,26,66,99,99,99,99],[24,26,56,99,99,99,99,99],[47,66,99,99,99,99,99,99],[99,99,99,99,99,99,99,99],[99,99,99,99,99,99,99,99],[99,99,99,99,99,99,99,99],[99,99,99,99,99,99,99,99]]))

In [4]:
def RGB2YCbCr(RGB):
  B=np.array([0,128,128])
  return(np.dot(np.array([[0.299,0.587,0.114],[-0.169,-0.331,0.500],[0.500,-0.419,-0.081]]),RGB))
def CbCr2RGB(Cb):
  B=np.array([0,128,128])
  return(np.dot(np.array([[1,0,1.402],[1,-0.344,-0.714],[1,1.772,0.0]]),Cb-B))

In [5]:
def DCT(image):
  N=image.shape
  img_t=np.zeros(N)
  for u in range(N[0]):
    for v in range(N[1]):
      if u==0:
        alfa_u=np.sqrt(1/N[0])
      else:
        alfa_u=np.sqrt(2/N[0])
      if v==0:
        alfa_v=np.sqrt(1/N[1])
      else:
        alfa_v=np.sqrt(2/N[1])
      for x in range(N[0]):
        for y in range(N[1]):
          img_t[u,v]=alfa_u*alfa_v*image[x,y]*np.cos((2*x+1)*u*np.pi/(2*N[0]))*np.cos((2*y+1)*v*np.pi/(2*N[1]))+img_t[u,v]
  return img_t

In [6]:
def zig_zag(mat):
  i=-1
  j=1
  ordered_mat=np.array([])
  ordered_mat=np.append(ordered_mat,mat[0,0])
  while(i<8 and j<8):
    if(i==-1):
      i+=1
      while(j>=0 and i<8 and j<8):
        ordered_mat=np.append(ordered_mat,mat[i,j])
        j=j-1
        i=i+1
    if(j==-1):
      j+=1
      while(i>=0 and i<8 and j<8):
        ordered_mat=np.append(ordered_mat,mat[i,j])
        i=i-1
        j=j+1
  j=j-1
  for k in range(7):
    if(i==8):
      i-=1
      j+=2
      while(j<8):
        ordered_mat=np.append(ordered_mat,mat[i,j])
        j=j+1
        i=i-1
    if(j==8):
      j-=1
      i+=2
      while(i<8):
        ordered_mat=np.append(ordered_mat,mat[i,j])
        j=j-1
        i=i+1
  ordered_mat=list(ordered_mat.astype(int))
  return(ordered_mat)

In [7]:
def huffcode(code_dict,node,code):

    if(node[1][1]==None and node[1][2]==None):
        code_dict[node[1][0]]=code
        return

    if(node[1][1]!=None):
        code_nxt=code+"0"
        huffcode(code_dict,node[1][1],code_nxt)

    if(node[1][2]!=None):
        code_nxt=code+"1"
        huffcode(code_dict,node[1][2],code_nxt)

def modify_huffcode(code_dict):
    new_code_dict = dict()
    vals_list = [[] for _ in range(17)]
    for (key,val) in code_dict.items():
        vals_list[len(val)].append(key)

    cur = 0
    for i in range(17):
        for j in range(len(vals_list[i])):
            cur_bin = bin(cur)[2:]
            if '0'*(i-len(cur_bin))+cur_bin=='1'*i:
                vals_list[i+1] = [vals_list[i][j]]+vals_list[i+1]
                continue
            new_code_dict[vals_list[i][j]]='0'*(i-len(cur_bin))+cur_bin
            cur+=1
        cur<<=1
    return new_code_dict

def huffcode_gen(x):
    freq=dict()
    for i in range(len(x)):
        if x[i] in freq:
            freq[x[i]]+=1
        else:
            freq[x[i]]=1

    pq=list()
    for ele in freq:
        pq.append([freq[ele],[ele,None,None]])

    pq.sort()
    while(len(pq)>1):
        t1=pq[0]
        t2=pq[1]

        curr=[t1[0]+t2[0],[-1,t1,t2]]
        pq.remove(t1)
        pq.remove(t2)
        pq.append(curr)
        pq.sort(key = lambda t:t[0])

    huffTree=pq[0]

    code_dict=dict()
    code=""
    huffcode(code_dict,huffTree,code)
    code_dict=modify_huffcode(code_dict)
    return code_dict

In [8]:
def write_Q(final_image, quant_table, type):
    final_image.write(b'\xff\xdb')
    final_image.write(pack(">H",67))
    final_image.write(pack("B",type))
    zigzag_quant = zig_zag(quant_table)
    for i in range(8*8):
        final_image.write(pack("B",zigzag_quant[i]))

In [9]:
def header(final_image,height,width,components):
    final_image.write(b'\xff\xc0')
    final_image.write(pack(">H",8+3*components))
    final_image.write(pack("B",8))
    final_image.write(pack(">HHB",height,width,components))
    for i in range(components):
        final_image.write(pack("BBB", i+1, 16+1, min(i,1)))

In [10]:
def write_huff_code(final_image,type,code):
    final_image.write(b'\xff\xc4')
    final_image.write(pack(">HB",19+len(code),type))
    val_list = [[] for _ in range(16)]
    for (key,val) in code.items():
        val_list[len(val)-1].append(key)
    for i in range(16):
        final_image.write(pack("B",len(val_list[i])))
    for i in range(16):
        for val in val_list[i]:
            final_image.write(pack("B",val))

In [11]:
def var_info(x):
    if x==0:
        return ''
    if x<0:
        x=-x
        ans = bin((~x)&((1<<int(x).bit_length())-1))[2:]
        ans = "0"*(len(bin(x))-2-len(ans))+ans
    else:
        ans=bin(x)[2:]
    return ans

In [12]:
def get_symb_val(seq):
    vlc,vli = [],[]
    for i in range(0,len(seq),3):
        vlc.append(((seq[i]<<4)+seq[i+1]))
        if vlc[-1]==0:
            break
        vli.append(var_info(seq[i+2]))
    return vlc,vli

In [13]:
def pad_image(image):
  N=image.shape
  while(N[0]%8!=0):
    image = np.insert(image,-1, image[N[0]-1,:,:], axis=0)
    N=image.shape
  while(N[1]%8!=0):
    image = np.insert(image,-1, image[:,N[1]-1,:], axis=1)
    N=image.shape
  return(image)

In [14]:
def compress(image):
    image=pad_image(image)
    image_t=np.zeros(image.shape)
    height=image.shape[0]
    width=image.shape[1]
    DC_y, DC_Cb, DC_Cr, AC_y, AC_Cb, AC_Cr = [],[],[],[],[],[]
    DC_y_vli,DC_Cb_vli,DC_Cr_vli,AC_Y_vli,AC_Cb_vli,AC_Cr_vli = [],[],[],[],[],[]
    quant_lum = Q_l()
    quant_chr = Q_c()
    image_d=np.zeros(image.shape)
    image_t=np.zeros(image.shape)
    for j in range(0,width):
        for i in range(0,height):
          image_t[i,j,2]=image[i,j,0]-128
          image_t[i,j,1]=image[i,j,1]-128
          image_t[i,j,0]=image[i,j,2]-128
    prev_Y_dc,prev_CB_dc,prev_CR_dc = 0,0,0
    for i in range(height):
      for j in range(width):
        image_t[i,j,:]=RGB2YCbCr(image_t[i,j,:])
    for i in range(0,image_t.shape[0],8):
      for j in range(0,image_t.shape[1],8):
        image_d[i:i+8,j:j+8,0]=DCT(image_t[i:i+8,j:j+8,0])/Q_l()
        image_d[i:i+8,j:j+8,1]=DCT(image_t[i:i+8,j:j+8,1])/Q_c()
        image_d[i:i+8,j:j+8,2]=DCT(image_t[i:i+8,j:j+8,2])/Q_c()
    image_d=image_d.astype(int)
    for i in range(height//8):
        for j in range(width//8):
            Y_z=zig_zag(image_d[i*8:i*8+8,j*8:j*8+8,0])
            CB_z=zig_zag(image_d[i*8:i*8+8,j*8:j*8+8,1])
            CR_z=zig_zag(image_d[i*8:i*8+8,j*8:j*8+8,2])
            z=Y_z
            p,q = 1, 1
            lst = 63
            rle = []
            while lst>=0 and z[lst]==0:
                lst -= 1
            while q<=lst:
                if (z[q]==0 and q-p+1==16) or z[q]!=0:
                    rle += [q-p, int(z[q]).bit_length(), z[q]]
                    p=q+1
                q += 1
            if lst!=63:
                rle += [0,0]
            Y_rle=rle
            z=CB_z
            p,q = 1, 1
            lst = 63
            rle = []
            while lst>=0 and z[lst]==0:
                lst -= 1
            while q<=lst:
                if (z[q]==0 and q-p+1==16) or z[q]!=0:
                    rle += [q-p, int(z[q]).bit_length(), z[q]]
                    p=q+1
                q += 1
            if lst!=63:
                rle += [0,0]
            CB_rle=rle
            z=CR_z
            p,q = 1, 1
            lst = 63
            rle = []
            while lst>=0 and z[lst]==0:
                lst -= 1
            while q<=lst:
                if (z[q]==0 and q-p+1==16) or z[q]!=0:
                    rle += [q-p, int(z[q]).bit_length(), z[q]]
                    p=q+1
                q += 1
            if lst!=63:
                rle += [0,0]
            CR_rle=rle
            Y_rl_class, Y_vli = get_symb_val(Y_rle)
            CB_rl_class, CB_vli = get_symb_val(CB_rle)
            CR_rl_class, CR_vli = get_symb_val(CR_rle)
            AC_y = AC_y+Y_rl_class
            AC_Cb = AC_Cb+CB_rl_class
            AC_Cr = AC_Cr+CR_rl_class
            AC_Y_vli = AC_Y_vli+Y_vli
            AC_Cb_vli = AC_Cb_vli+CB_vli
            AC_Cr_vli = AC_Cr_vli+CR_vli

            Y_dc,CB_dc,CR_dc = Y_z[0]-prev_Y_dc, CB_z[0]-prev_CB_dc, CR_z[0]-prev_CR_dc
            DC_y.append(int(Y_dc).bit_length())
            DC_Cb.append(int(CB_dc).bit_length())
            DC_Cr.append(int(CR_dc).bit_length())
            DC_y_vli.append(var_info(Y_dc))
            DC_Cb_vli.append(var_info(CB_dc))
            DC_Cr_vli.append(var_info(CR_dc))
            prev_Y_dc,prev_CB_dc,prev_CR_dc = Y_z[0],CB_z[0],CR_z[0]
    for i in range(len(AC_y)):
      if AC_y[i]>255:
        AC_y[i]=255
    lum_dc_code = huffcode_gen(DC_y)
    lum_ac_code = huffcode_gen(AC_y)
    chr_dc_code = huffcode_gen(DC_Cb+DC_Cr)
    chr_ac_code = huffcode_gen(AC_Cb+AC_Cr)

    output_file = input("Enter output file name: ")
    final_image = open(output_file+".jpg", "wb")
    final_image.write(b'\xff\xd8\xff\xe0')
    final_image.write(pack(">H",16))
    final_image.write(b'JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00')
    write_Q(final_image, quant_lum, 0)
    write_Q(final_image, quant_chr, 1)
    header(final_image,height,width,3)
    write_huff_code(final_image,0,lum_dc_code)
    write_huff_code(final_image,16,lum_ac_code)
    write_huff_code(final_image,1,chr_dc_code)
    write_huff_code(final_image,17,chr_ac_code)
    components=3
    final_image.write(b'\xff\xda')
    final_image.write(pack(">HB",6+2*components,components))
    final_image.write(b'\x01\x00\x02\x11\x03\x11')
    final_image.write(pack("BBB",0,63,0))
    Y_dc_ptr, CB_dc_ptr, CR_dc_ptr, Y_ac_ptr, CB_ac_ptr, CR_ac_ptr,Y_dc_vli_ptr, CB_dc_vli_ptr, CR_dc_vli_ptr, Y_ac_vli_ptr, CB_ac_vli_ptr, CR_ac_vli_ptr = 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0
    data = ""
    while Y_dc_ptr<len(DC_y):
      data = data+lum_dc_code[DC_y[Y_dc_ptr]]+DC_y_vli[Y_dc_vli_ptr]
      Y_dc_ptr+=1
      Y_dc_vli_ptr+=1
      size = 1
      while size<8*8 and AC_y[Y_ac_ptr]!=0:
          data = data+lum_ac_code[AC_y[Y_ac_ptr]]+AC_Y_vli[Y_ac_vli_ptr]

          size += (AC_y[Y_ac_ptr]>>4)
          size += 1
          Y_ac_ptr+=1
          Y_ac_vli_ptr+=1
      if size<8*8:
          data = data+lum_ac_code[0]
          Y_ac_ptr+=1

      data = data+chr_dc_code[DC_Cb[CB_dc_ptr]]+DC_Cb_vli[CB_dc_vli_ptr]
      CB_dc_ptr+=1
      CB_dc_vli_ptr+=1
      size = 1
      while size<8*8 and AC_Cb[CB_ac_ptr]!=0:
          data = data+chr_ac_code[AC_Cb[CB_ac_ptr]]+AC_Cb_vli[CB_ac_vli_ptr]
          size += (AC_Cb[CB_ac_ptr]>>4)
          size += 1
          CB_ac_ptr+=1
          CB_ac_vli_ptr+=1
      if size<8*8:
          data = data+chr_ac_code[0]
          CB_ac_ptr+=1
      data = data+chr_dc_code[DC_Cr[CR_dc_ptr]]+DC_Cr_vli[CR_dc_vli_ptr]
      CR_dc_ptr+=1
      CR_dc_vli_ptr+=1
      size = 1
      while size<8*8 and AC_Cr[CR_ac_ptr]!=0:
          data = data+chr_ac_code[AC_Cr[CR_ac_ptr]]+AC_Cr_vli[CR_ac_vli_ptr]
          size += (AC_Cr[CR_ac_ptr]>>4)
          size += 1
          CR_ac_ptr+=1
          CR_ac_vli_ptr+=1
      if size<8*8:
          data = data+chr_ac_code[0]
          CR_ac_ptr+=1
    while len(data)%8!=0:
            data=data+"0"
    result = ""
    for i in range(0,len(data),8):
            temp = data[i:i+8]
            result = result+temp
            if temp=="11111111":
                result = result+"00000000"
    data = result
    data=bitarray(data)
    final_image.write(data)
    final_image.write(b'\xff\xd9')
    final_image.close()


In [15]:
name=input('Enter the name of the file:   ')
image = cv2.imread(name)
compress(image)

Enter the name of the file:   corn.bmp
Enter output file name: oo
